# STEP 1 — DATA INTEGRATION & STRUCTURAL VALIDATION

### Objective:
- Load and merge transaction + identity datasets
- Validate data integrity and structure
- Prepare integrated dataset for analysis

In [10]:
# Import Libraries
import pandas as pd
import numpy as np

In [11]:
# Step 1A: Load Raw Datasets


print("=" * 60)
print("STEP 1A: Loading Raw Datasets")
print("=" * 60)

train_tx = pd.read_csv("../data/raw/train_transaction.csv")
train_id = pd.read_csv("../data/raw/train_identity.csv")

print(f"Transaction dataset shape: {train_tx.shape}")
print(f"Identity dataset shape: {train_id.shape}")
print()

STEP 1A: Loading Raw Datasets
Transaction dataset shape: (590540, 394)
Identity dataset shape: (144233, 41)



In [12]:
# Step 1B: Validate Data Integrity


print("=" * 60)
print("STEP 1B: Data Integrity Validation")
print("=" * 60)

# Check TransactionID uniqueness
try:
    assert train_tx["TransactionID"].is_unique, "Transaction IDs not unique in train_transaction!"
    assert train_id["TransactionID"].is_unique, "Transaction IDs not unique in train_identity!"
    print("✓ TransactionID is unique in both datasets")
except AssertionError as e:
    print(f"❌ CRITICAL ERROR: {e}")
    raise

# Check for missing TransactionIDs
print(f"✓ No missing TransactionIDs in train_transaction: {train_tx['TransactionID'].isna().sum() == 0}")
print(f"✓ No missing TransactionIDs in train_identity: {train_id['TransactionID'].isna().sum() == 0}")
print()

STEP 1B: Data Integrity Validation
✓ TransactionID is unique in both datasets
✓ No missing TransactionIDs in train_transaction: True
✓ No missing TransactionIDs in train_identity: True



In [13]:
# Step 1C: Merge Datasets


print("=" * 60)
print("STEP 1C: Merging Datasets")
print("=" * 60)

train = train_tx.merge(
    train_id,
    on="TransactionID",
    how="left"
)

print(f"Merged dataset shape: {train.shape}")
print(f"Expected shape: ({train_tx.shape[0]}, {train_tx.shape[1] + train_id.shape[1] - 1})")
print()

# Validate merge
assert train.shape[0] == train_tx.shape[0], "Row count mismatch after merge!"
assert train.shape[1] == train_tx.shape[1] + train_id.shape[1] - 1, "Column count mismatch after merge!"
print("✓ Merge validation passed")
print()

STEP 1C: Merging Datasets


Merged dataset shape: (590540, 434)
Expected shape: (590540, 434)

✓ Merge validation passed



In [14]:
# Step 1D: Target Variable Analysis


print("=" * 60)
print("STEP 1D: Target Variable Analysis")
print("=" * 60)

print("Fraud distribution (counts):")
print(train["isFraud"].value_counts())
print()

print("Fraud distribution (proportions):")
fraud_dist = train["isFraud"].value_counts(normalize=True)
print(fraud_dist)
print()

fraud_rate = train["isFraud"].mean()
print(f"📊 Fraud rate: {fraud_rate:.2%}")
print(f"   Class imbalance ratio: 1:{int(1/fraud_rate)}")
print()

STEP 1D: Target Variable Analysis
Fraud distribution (counts):
isFraud
0    569877
1     20663
Name: count, dtype: int64

Fraud distribution (proportions):
isFraud
0    0.96501
1    0.03499
Name: proportion, dtype: float64

📊 Fraud rate: 3.50%
   Class imbalance ratio: 1:28



In [15]:
# Step 1E: Identity Coverage Analysis


print("=" * 60)
print("STEP 1E: Identity Coverage Analysis")
print("=" * 60)

# Get identity columns (excluding TransactionID)
identity_cols = train_id.columns.drop("TransactionID").tolist()

print(f"Total identity features: {len(identity_cols)}")
print()

# Calculate rate of completely missing identity info
missing_identity_rate = train[identity_cols].isna().all(axis=1).mean()

print(f"Transactions with NO identity info: {missing_identity_rate:.2%}")
print(f"Transactions with SOME identity info: {1 - missing_identity_rate:.2%}")
print()

# Identity feature completeness
identity_completeness = (1 - train[identity_cols].isna().mean()).sort_values()
print("Identity feature completeness (top 10 most complete):")
print(identity_completeness.tail(10))
print()

STEP 1E: Identity Coverage Analysis
Total identity features: 40

Transactions with NO identity info: 75.58%
Transactions with SOME identity info: 24.42%

Identity feature completeness (top 10 most complete):
id_28    0.238727
id_29    0.238727
id_11    0.238727
id_15    0.238739
id_35    0.238739
id_36    0.238739
id_37    0.238739
id_38    0.238739
id_12    0.244239
id_01    0.244239
dtype: float64



In [16]:
# Step 1F: Data Quality Summary


print("=" * 60)
print("STEP 1F: Data Quality Summary")
print("=" * 60)

# Missing values overview
missing_summary = train.isna().sum().sort_values(ascending=False)
missing_top = missing_summary[missing_summary > 0].head(10)

print(f"Total columns: {train.shape[1]}")
print(f"Columns with missing values: {(missing_summary > 0).sum()}")
print(f"Total missing values: {train.isna().sum().sum():,}")
print()

print("Top 10 columns with most missing values:")
print(missing_top)
print()

# Memory usage
memory_mb = train.memory_usage(deep=True).sum() / 1024**2
print(f"Memory usage: {memory_mb:.2f} MB")
print()

STEP 1F: Data Quality Summary
Total columns: 434
Columns with missing values: 414
Total missing values: 115,523,073

Top 10 columns with most missing values:
id_24    585793
id_25    585408
id_07    585385
id_08    585385
id_21    585381
id_26    585377
id_27    585371
id_23    585371
id_22    585371
dist2    552913
dtype: int64

Memory usage: 2567.09 MB



In [17]:
# Step 1G: Column Type Summary


print("=" * 60)
print("STEP 1G: Column Type Summary")
print("=" * 60)

numeric_cols = train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = train.select_dtypes(include=["object"]).columns.tolist()

print(f"Numeric columns: {len(numeric_cols)}")
print(f"Categorical columns: {len(categorical_cols)}")
print()

print("Sample categorical columns:")
print(categorical_cols[:10])
print()


STEP 1G: Column Type Summary
Numeric columns: 403
Categorical columns: 31

Sample categorical columns:
['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5']



In [18]:
# Step 1H: Save Merged Dataset


print("=" * 60)
print("STEP 1H: Saving Merged Dataset")
print("=" * 60)

output_path = "../data/raw/train_merged.parquet"
train.to_parquet(output_path, index=False)

print(f"✓ Saved to: {output_path}")
print(f"✓ Shape: {train.shape}")
print(f"✓ Size: {memory_mb:.2f} MB")
print("=" * 60)
print("\n🎯 Step 1 completed successfully!")
print("   Next: Step 2 - Data Audit & Risk Profiling")
print("=" * 60)

STEP 1H: Saving Merged Dataset
✓ Saved to: ../data/raw/train_merged.parquet
✓ Shape: (590540, 434)
✓ Size: 2567.09 MB

🎯 Step 1 completed successfully!
   Next: Step 2 - Data Audit & Risk Profiling
